**Question:** Load, split, preprocess, and build the LSTM model using the sunspot dataset

**Description:**

The dataset consists from 1749/01/01 to 2017/08/31 

Load the dataset using pandas 

Split the dataset into training till 3000 rows and testing sets as remaining 

By using helper function turn data into a window dataset

For inference, we just need to convert the data into multiple samples of predictor variables.

For input, we are converting the time series into samples of 60 (window_size). The first 59 data points of a sample will be used as the predictor variables while the last data point will be used as the target variable.

Build a sequential model where input layer is Conv1D with 60 filters, kernel size as 5, relu as activation. Add 2 layers of lstm layers with 60 neurons each. Add two dense layers where 30 and 10 neurons respectively. Finally, add lambda layer for scaling output to same range of values

Print total model parameters




**Level** : Medium


**Input format:**
Dataset


**Output format:**
Model parameters


**Sample Input** :
Sunspot dataset


**Sample Output** : 
60591


**Solution:**



In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np

df = pd.read_csv('/home/meQuestionogy/Sunspots.csv', usecols=['Date', 'Monthly Mean Total Sunspot Number'])

time = np.array(list(df.index))
sunspots = list(df['Monthly Mean Total Sunspot Number'])
series = np.array(sunspots)

t_train = time[:3000]
train = series[:3000]
t_test = time[3000:]
test = series[3000:]

window_size = 60  
batch_size = 32
shuffle_buffer_size = 1000
forecast_period = 30  

def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)  
    s = tf.data.Dataset.from_tensor_slices(series)
    s = s.window(window_size, shift=1, drop_remainder=True)  
    s = s.flat_map(lambda i: i.batch(window_size))
    s = s.map(lambda i: (i[:-1], i[-1:]))  
    s = s.shuffle(shuffle_buffer)  
    s = s.batch(batch_size).prefetch(1) 
    return s

def model_forecast(model, series, window_size, batch_size):
    s = tf.data.Dataset.from_tensor_slices(series)
    s = s.window(window_size, shift=1, drop_remainder=True)  
    s = s.flat_map(lambda w: w.batch(window_size))
    s = s.batch(batch_size).prefetch(1)
    forecast = model.predict(s)
    return forecast

tf.keras.backend.clear_session()
tf.random.set_seed(51)
np.random.seed(51)
training = windowed_dataset(train, window_size=window_size,batch_size=batch_size, shuffle_buffer=shuffle_buffer_size)

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(filters=60, kernel_size=5,strides=1, padding="causal",activation="relu",input_shape=[None, 1]), 
  tf.keras.layers.LSTM(60, activation="tanh", return_sequences=True),
  tf.keras.layers.LSTM(60, activation="tanh", return_sequences=False),
  tf.keras.layers.Dense(30, activation="relu"),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(1),
  tf.keras.layers.Lambda(lambda x: x * 100)  
])

print(model.count_params())
